## Watch Reviews

In [97]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 3s (34.1 kB/s)
Reading package lists... Done


In [98]:
# Install postgressql 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-21 00:09:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-11-21 00:09:23 (1.26 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [99]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("WatchReviews").getOrCreate()

In [100]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

Change Datatype for functionality

In [101]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [102]:
# Next we need to create the list of struct fields
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", IntegerType(), True), StructField("review_id", StringType(), True), StructField("product_id", StringType(), True), StructField("product_parent", IntegerType(), True), StructField("product_title", StringType(), True), StructField("product_category", StringType(), True), StructField("star_rating", IntegerType(), True), StructField("helpful_votes", IntegerType(), True), StructField("total_votes", IntegerType(), True), StructField("vine", StringType(), True), StructField("verified_purchase", StringType(), True), StructField("review_headline", StringType(), True), StructField("review_body", StringType(), True), StructField("review_date", StringType(), True)]
schema

[StructField(marketplace,StringType,true),
 StructField(customer_id,IntegerType,true),
 StructField(review_id,StringType,true),
 StructField(product_id,StringType,true),
 StructField(product_parent,IntegerType,true),
 StructField(product_title,StringType,true),
 StructField(product_category,StringType,true),
 StructField(star_rating,IntegerType,true),
 StructField(helpful_votes,IntegerType,true),
 StructField(total_votes,IntegerType,true),
 StructField(vine,StringType,true),
 StructField(verified_purchase,StringType,true),
 StructField(review_headline,StringType,true),
 StructField(review_body,StringType,true),
 StructField(review_date,StringType,true)]

In [103]:
# Pass in our fields
final = StructType(fields=schema)
final

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,StringType,true)))

In [104]:
# Read our data with our new schema
table = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), schema=final, sep="\t", header=True)
table = table.withColumn("review_date", table["review_date"].cast("date"))
table.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [105]:
df = table.select(["star_rating", "helpful_votes", "total_votes", "review_body", "review_date"])
df.show(5)

+-----------+-------------+-----------+--------------------+-----------+
|star_rating|helpful_votes|total_votes|         review_body|review_date|
+-----------+-------------+-----------+--------------------+-----------+
|          5|            0|          0|Absolutely love t...| 2015-08-31|
|          5|            0|          0|I love this watch...| 2015-08-31|
|          2|            1|          1|           Scratches| 2015-08-31|
|          5|            0|          0|It works well on ...| 2015-08-31|
|          4|            0|          0|Beautiful watch f...| 2015-08-31|
+-----------+-------------+-----------+--------------------+-----------+
only showing top 5 rows



In [106]:
# Print our schema same idea as "dtypes" in pandas
# Shows us the data types of the columns *AND* they can be changed.
df.printSchema()

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



Use Pyspark to build Pipeline in order to Train Test written reviews with Bayes Theorem

In [107]:
# Import the required dependencies
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import length
from pyspark.sql import functions as f

Create a positive review class and a negative review class

In [108]:
df = df.where(df.star_rating != 3)
df = df.withColumn('class', f.when(f.col('star_rating') > 3, "positive").otherwise("negative"))

#df = df.withColumn(("class", "positive").where(df.star_rating > 3) and ("class", "negative").where(df.star_rating < 3))
df.show()

+-----------+-------------+-----------+--------------------+-----------+--------+
|star_rating|helpful_votes|total_votes|         review_body|review_date|   class|
+-----------+-------------+-----------+--------------------+-----------+--------+
|          5|            0|          0|Absolutely love t...| 2015-08-31|positive|
|          5|            0|          0|I love this watch...| 2015-08-31|positive|
|          2|            1|          1|           Scratches| 2015-08-31|negative|
|          5|            0|          0|It works well on ...| 2015-08-31|positive|
|          4|            0|          0|Beautiful watch f...| 2015-08-31|positive|
|          5|            0|          0|i love this watch...| 2015-08-31|positive|
|          5|            1|          1|for my wife and s...| 2015-08-31|positive|
|          1|            1|          5|I was about to bu...| 2015-08-31|negative|
|          5|            1|          2|Watch is perfect....| 2015-08-31|positive|
|          4|   

In [109]:
# Create a column for length of body text
df = df.withColumn("length", length(df["review_body"]))
df.show(5)

+-----------+-------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------+------+
|star_rating|helpful_votes|total_votes|review_body                                                                                                                                                                                                                                                                                                                           

In [119]:
df = df.select(["class", "review_body", "length"])
df.show()

+--------+--------------------+------+
|   class|         review_body|length|
+--------+--------------------+------+
|positive|Absolutely love t...|    80|
|positive|I love this watch...|    44|
|negative|           Scratches|     9|
|positive|It works well on ...|    94|
|positive|Beautiful watch f...|   566|
|positive|i love this watch...|   127|
|positive|for my wife and s...|    60|
|negative|I was about to bu...|   244|
|positive|Watch is perfect....|   322|
|positive|Great quality and...|   145|
|positive|The watch was pre...|   550|
|negative|I bought this wat...|   446|
|positive|Heavier than i th...|    21|
|positive|Had it for severa...|   144|
|positive|This one is diffe...|   247|
|positive|The watch is attr...|   134|
|positive|      said my wife..|    14|
|positive|Nice watch, on ti...|    41|
|positive|Looks great and l...|   225|
|positive|I really like thi...|   269|
+--------+--------------------+------+
only showing top 20 rows



In [120]:
# Import pipeline dependencies
from pyspark.ml.feature import StringIndexer as SI
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover as stop
from pyspark.ml.feature import HashingTF as hash
from pyspark.ml.feature import IDF

In [126]:
# Create all the features to the data set to pipeline
index = SI(inputCol="class", outputCol="target")
tokenizer = Tokenizer(inputCol="review_body", outputCol="token_text")
stopper = stop(inputCol="token_text", outputCol="stop_tokens")
hasher = hash(inputCol="stop_tokens", outputCol="hash_token")
idf = IDF(inputCol="hash_token", outputCol="idf_token")

In [127]:
from pyspark.ml.feature import VectorAssembler as VA
# from pyspark.ml.linalg import Vector

# Create feature vectors
assemble = VA(inputCols=["idf_token", "length"], outputCol="features")

In [128]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
pipeline_steps = Pipeline(stages=[index, tokenizer, stopper, hasher, idf, assemble])

In [129]:
# Fit and transform the pipeline
cleaner = pipeline_steps.fit(df)
cleaned = cleaner.transform(df)

Py4JJavaError: ignored

I do not know why this cell is not working. With the same format, it did work in a previous example

In [ ]:
# Show label and pipeline result features
cleaned_df = cleaned.select(["target", "features"])
cleaned_df.show(truncate=False)

In [ ]:
# Break into training and testing sets

# Import dependency 
from pyspark.ml.classification import NaiveBayes as nb

# Break data data into training set and test set
train, test = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit the training data
predictor = nb.fit(train)

# Transform the model with the test data
test_results = predictor.transform(test)
test_results.show(5)

In [ ]:
# Use pyspark to Evalute Results

# Import Evaluator tool dependency
from pyspark.ml.evaluation import MultiClassificationEvaluator as mce

accuracy=mce.evalutate(test_results)
print("Accuracy of model at predicting reviews was :%f" % accuracy)

Results would be super awesome to analyze here

### Accessing data

In [ ]:
dataframe['price']

In [ ]:
type(dataframe['price'])

In [ ]:
dataframe.select('price')

In [ ]:
type(dataframe.select('price'))

In [ ]:
dataframe.select('price').show()

### Manipulating Columns

In [ ]:
# Add new column
dataframe.withColumn('newprice', dataframe['price']).show()

In [ ]:
# Update column name
dataframe.withColumnRenamed('price','newerprice').show()

In [ ]:
# Double the price
dataframe.withColumn('doubleprice',dataframe['price']*2).show()

In [ ]:
# Add a dollar to the price
dataframe.withColumn('add_one_dollar',dataframe['price']+1).show()

In [ ]:
# Half the price
half_df = dataframe.withColumn('half_price',dataframe['price']/2)
half_df.show()

In [ ]:
# Collecting a column as a list
dataframe.select("price").collect()

In [ ]:
half_df.select(avg("half_price"), avg("price")).show(truncate=False)

In [ ]:
# Tokenize / split to individual words

# import first
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="<-column with sentence here->", outputCol="<-new token *words* column here->") # Change
# transform and show data
tokenized = tokenizer.transform(dataframe)
tokenized.show(truncate=False)

In [ ]:
# Import UDF tool
from pyspark.sql.functions import col, udf

# Create a function
def list_length(word_list):
  return len(word_list)  ## Note length is a built in function so we might not use this "withColumn" tool

# Create UDF (user defined function)
token_count = udf(list_length, IntegerType())
count_tokenized = tokenized.select("<-column with sentence here->", "<-new *words*-here").withColumn("<-token count here->", token_count(col("<new*words")))
count_tokenized.show(truncate=False)

In [ ]:
# Import StopWords ***** Probably want to do this before Tokenizing *****
from pyspark.ml.feature import StopWordsRemover as stop

# Instantiate the remover
remove = stop(inputCol="column with sentence here->", outputCol="<-filtered old column->")
filtered_df = remove.transform(dataframe)
filtered_df.show(truncate=False)

In [ ]:
# TF IDF Term Frequency / Inverse Document Frequency
# 2 to the 4th can be adjusted upward as needed
# OR allow it to default by not specifying ( 262145 buckets )

# Import Hashing Term Frequency tool
from pyspark.ml.feature import HashingTF as hash
hashing = hash(inpupCol="<-filtered old column here->", outputCol="<- new*values*here->", numFeatures=pow(2,4)) # 2 to the 4th

# Transform new column into the dataframe and show
hashed_df = hashing.transform(filtered_df)
hashed_df.show(truncate=False)

In [ ]:
# Fit transform to the data
idf = IDF(inputCol="<-new*values*here->", "features")
idfModel = idf.fit(hashed_df)
rescaleData = idfModel.transform(hashed_df)

# Display the scaled data
scaled_df = rescaleData.select("<-filtered*words*column-here->", "features")
scaled_df.show(truncate=False)

In [ ]:
# Sample Reference df.show()

## This class example df name had
# col 1 a pos/neg label column called "class**"
# col 2 a sentence column called "text**"
# col 3 a length column called "length**"

In [ ]:
# Use pyspark to ..somethinghere... Bayes Theorem

# Import String Indexer tool
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector # this is actaully not needed here
from pyspark.ssql.functions import length ## Or use the function code from above?

# Create all the features
pos_neg_to_num = StringIndexer(inputCol="<-class**_orwhatever->", outputCol="label") # could also call label "target?"

## Mabye some of this also from above
tokenizer = Tokenizer(inputCol="text**_orwhatever", outputCol="token_text_orwhatever")
stopremove = stop(inputCol="token_text_orwhatever", outputCol="stop_tokens_orwhatever")
hashingTF = hash(inputCol="stop_tokens_orwhatever", outputCol="hash_token_orwhatever")
idf = IDF(inputCol="hash_token_orwhatever", outputCol="idf_token_orwhatever")

# Clean Up
clean_up = VectorAssembler(inputCol=["idf_token_orwhatever", "length**_orwhatever"], outputCol="features")

# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, cleanup])

# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df)
cleaned = cleaner.transform(df)

# Show label and pipeline result features
cleaned_df = cleaned.select(["label", "features"])
cleaned_df.show(truncate=False)
